In [1]:
import utils.dataset_utils as dataset
import utils.train_utils as train

import numpy as np
import copy
import pickle

from sklearn.base import BaseEstimator, ClassifierMixin
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from datetime import datetime
from dateutil.relativedelta import relativedelta

In [2]:
def unit_vector_norm(X):
    return (X.T / np.sqrt((X**2).sum(axis=1))).T

In [3]:
file_location = "../data/Raman_Mouse/preprocess/"
filenames = np.load(f"{file_location}FileNames.npy")
with open(f'{file_location}Sample_labels.pickle', 'rb') as f:
    labels = pickle.load(f)

data = []
for f in filenames:
    x = np.load(f"{file_location}{f.split('.')[0]}_raman.npy")
    data.append((unit_vector_norm(x.reshape(-1,x.shape[-1])).reshape(x.shape), labels[f]))


In [4]:
class MLP(nn.Module):
    def __init__(self, size=1300*7*7 , output=10, depth=2, neurons=1300, bias=True, **kwargs):
        super().__init__()

        self.layers = nn.Sequential( 
            nn.Dropout3d(0.25),
            nn.Flatten(),
            nn.Linear(size, neurons, bias=bias),
            nn.ReLU(True),
            *((nn.Linear(neurons, neurons, bias=bias),
            nn.ReLU(True)) * (depth-1)),
            nn.Linear(neurons, output, bias=bias)
        )
  
    def forward(self, x):
        return self.layers(x)

In [5]:
class SupervisedClassifier(BaseEstimator):
    def __init__(self, **kwargs):
        self.kwargs = kwargs

        _use_cuda = torch.cuda.is_available() and kwargs['cuda']
        if _use_cuda:
            torch.backends.cudnn.enabled = True
            torch.backends.cudnn.benchmark = True
        self.device = torch.device('cuda' if _use_cuda else 'cpu')        
        
    def fit(self, data):
        self.model = MLP(**self.kwargs).to(self.device)

        parameters = filter(lambda x: x.requires_grad, self.model.parameters())
        self.optimizer = optim.Adam(parameters)        
        train_loader, test_loader = dataset.load_liver(data, self.kwargs['batch_size'], self.kwargs['patch_size'])
        
        for epoch in range(self.kwargs['epochs']):
            print('-'*50)
            print('Epoch {:3d}/{:3d}'.format(epoch+1, self.kwargs['epochs']))
            start_time = datetime.now()
            train.train(self.model, self.optimizer, train_loader, self.kwargs['loss_func'], self.kwargs['log_step'], self.device)
            end_time = datetime.now()
            time_diff = relativedelta(end_time, start_time)
            print('Elapsed time: {}h {}m {}s'.format(time_diff.hours, time_diff.minutes, time_diff.seconds))
            loss = train.test(self.model, test_loader, self.kwargs['loss_func'], self.device)
            print('Validation| bits: {:2.2f}'.format(loss), flush=True)    
                         
        return self
    
    def predict(self, X):
        """
        predict transforms the data into the reference space. Min weight should be 0 or higher then 'min_weight'
        The error is the NMSE, where the MSE is normalised by the signal strength. 
        error.shape = X.shape[0], so for each data point the error is calculated.
        """
        # Check is fit had been called
        check_is_fitted(self)

        # Input validation
        X = check_array(X)
#         self.model(X)
        
#         return RCA_vector

In [6]:
BATCH_SIZE = 64
EPOCHS = 10
PATCH_SIZE = 7

kwargs = {'size': PATCH_SIZE*PATCH_SIZE*data[0][0].shape[-1],
          'batch_size': BATCH_SIZE,
          'patch_size': PATCH_SIZE,
          'cuda': False,
          'log_step': 30,
          'epochs': 20,
          'depth': 3,
          'output': 4,
          'loss_func': nn.BCEWithLogitsLoss(),
          'neurons': data[0][0].shape[-1],
          'bias': True
         }

In [7]:
rvc = SupervisedClassifier(**kwargs)
rvc.fit(data)


--------------------------------------------------
Epoch   1/ 20
tensor([[-0.0312, -0.0219,  0.0256,  0.0134],
        [-0.0320, -0.0222,  0.0227,  0.0134],
        [-0.0324, -0.0221,  0.0222,  0.0127],
        [-0.0318, -0.0231,  0.0252,  0.0139],
        [-0.0325, -0.0222,  0.0225,  0.0134],
        [-0.0325, -0.0227,  0.0226,  0.0134],
        [-0.0312, -0.0219,  0.0256,  0.0134],
        [-0.0320, -0.0223,  0.0228,  0.0133],
        [-0.0312, -0.0219,  0.0256,  0.0134],
        [-0.0322, -0.0224,  0.0225,  0.0130],
        [-0.0319, -0.0222,  0.0228,  0.0132],
        [-0.0326, -0.0229,  0.0249,  0.0140],
        [-0.0323, -0.0222,  0.0227,  0.0135],
        [-0.0312, -0.0219,  0.0256,  0.0134],
        [-0.0322, -0.0224,  0.0225,  0.0131],
        [-0.0326, -0.0231,  0.0244,  0.0138],
        [-0.0322, -0.0224,  0.0225,  0.0132],
        [-0.0324, -0.0231,  0.0249,  0.0136],
        [-0.0324, -0.0228,  0.0246,  0.0132],
        [-0.0323, -0.0229,  0.0251,  0.0136],
        [-0.032

tensor([[0.0889, 0.0750, 0.1054, 0.0871],
        [0.0889, 0.0750, 0.1054, 0.0871],
        [7.7969, 4.4197, 5.7475, 5.0401],
        [7.8118, 4.4270, 5.7565, 5.0496],
        [7.8470, 4.6005, 5.9234, 5.0869],
        [7.7788, 4.5599, 5.8721, 5.0426],
        [0.0889, 0.0750, 0.1054, 0.0871],
        [7.5344, 4.2756, 5.5593, 4.8844],
        [0.0889, 0.0750, 0.1054, 0.0871],
        [0.0889, 0.0750, 0.1054, 0.0871],
        [7.4039, 4.3433, 5.5934, 4.8026],
        [7.0188, 4.1175, 5.3066, 4.5525],
        [8.0098, 4.5473, 5.9089, 5.1920],
        [0.0889, 0.0750, 0.1054, 0.0871],
        [8.2575, 4.6824, 6.0826, 5.3504],
        [7.9839, 4.5248, 5.8831, 5.1607],
        [8.3033, 4.7145, 6.1247, 5.3779],
        [0.0889, 0.0750, 0.1054, 0.0871],
        [8.0674, 4.5735, 5.9450, 5.2182],
        [7.8547, 4.4532, 5.7901, 5.0798],
        [9.1910, 5.2202, 6.7733, 5.9555],
        [0.0889, 0.0750, 0.1054, 0.0871],
        [6.9851, 4.0987, 5.2821, 4.5305],
        [0.0889, 0.0750, 0.1054, 0

tensor([[62.7948, 38.1566, 47.9348, 41.1528],
        [60.8636, 36.7528, 46.2205, 39.8843],
        [59.1350, 35.7095, 44.9115, 38.7457],
        [ 0.2377,  0.1848,  0.2246,  0.1851],
        [63.8133, 38.5335, 48.4563, 41.8155],
        [59.0623, 35.6652, 44.8559, 38.6981],
        [65.8169, 39.7602, 49.9922, 43.1359],
        [ 0.2377,  0.1848,  0.2246,  0.1851],
        [62.4197, 37.6929, 47.3997, 40.9025],
        [54.7323, 33.2699, 41.7975, 35.8745],
        [63.6987, 38.4658, 48.3712, 41.7405],
        [66.4362, 40.3574, 50.7045, 43.5289],
        [64.0624, 38.6819, 48.6428, 41.9776],
        [68.8533, 41.8245, 52.5449, 45.1285],
        [64.2678, 38.8015, 48.7971, 42.0936],
        [61.2592, 37.2260, 46.7667, 40.1482],
        [60.4183, 36.7200, 46.1297, 39.5988],
        [62.6716, 37.8402, 47.5880, 41.0662],
        [66.6082, 40.4612, 50.8345, 43.6388],
        [57.6638, 34.8221, 43.7948, 37.7833],
        [67.5750, 40.8035, 51.3082, 44.2774],
        [57.5925, 34.7771, 43.7399

tensor([[259.0167, 165.3700, 198.2286, 174.3001],
        [  0.5821,   0.4200,   0.4989,   0.4204],
        [261.5109, 166.9745, 200.1476, 175.9965],
        [  0.5821,   0.4200,   0.4989,   0.4204],
        [276.3591, 176.4507, 211.5096, 185.9797],
        [  0.5821,   0.4200,   0.4989,   0.4204],
        [268.9812, 171.7274, 205.8513, 180.9963],
        [279.5246, 178.7825, 214.2788, 188.0629],
        [258.9549, 165.3373, 198.1900, 174.2741],
        [243.7933, 155.6561, 186.5923, 164.0684],
        [  0.5821,   0.4200,   0.4989,   0.4204],
        [263.1652, 168.0228, 201.4089, 177.1045],
        [238.0919, 152.0168, 182.2275, 160.2357],
        [252.9657, 161.8168, 193.9427, 170.1959],
        [259.2060, 165.5012, 198.3850, 174.4444],
        [  0.5821,   0.4200,   0.4989,   0.4204],
        [241.9175, 154.4594, 185.1563, 162.8080],
        [259.6564, 165.7849, 198.7278, 174.7455],
        [270.9294, 172.9889, 207.3580, 182.3314],
        [256.2338, 163.9080, 196.4486, 172.3953],


tensor([[7.0168e+02, 4.8400e+02, 5.4427e+02, 4.9501e+02],
        [8.2603e+02, 5.6945e+02, 6.4024e+02, 5.8283e+02],
        [8.1111e+02, 5.5945e+02, 6.2911e+02, 5.7223e+02],
        [8.9901e+02, 6.2006e+02, 6.9726e+02, 6.3425e+02],
        [1.3228e+00, 9.6563e-01, 1.0837e+00, 9.6171e-01],
        [7.9045e+02, 5.4492e+02, 6.1267e+02, 5.5773e+02],
        [1.3228e+00, 9.6563e-01, 1.0837e+00, 9.6171e-01],
        [1.3228e+00, 9.6563e-01, 1.0837e+00, 9.6171e-01],
        [8.8428e+02, 6.0991e+02, 6.8585e+02, 6.2384e+02],
        [1.0730e+03, 7.3998e+02, 8.3206e+02, 7.5706e+02],
        [7.4541e+02, 5.1415e+02, 5.7818e+02, 5.2586e+02],
        [7.1566e+02, 4.9364e+02, 5.5511e+02, 5.0488e+02],
        [1.3228e+00, 9.6563e-01, 1.0837e+00, 9.6171e-01],
        [8.2269e+02, 5.6715e+02, 6.3766e+02, 5.8046e+02],
        [7.9032e+02, 5.4481e+02, 6.1255e+02, 5.5761e+02],
        [7.7980e+02, 5.3786e+02, 6.0485e+02, 5.5010e+02],
        [8.5666e+02, 5.9056e+02, 6.6398e+02, 6.0444e+02],
        [7.412

...

tensor([[1.3402e+09, 1.3148e+09, 1.3218e+09, 1.2974e+09],
        [1.3760e+09, 1.3499e+09, 1.3571e+09, 1.3320e+09],
        [1.3934e+06, 1.3670e+06, 1.3742e+06, 1.3489e+06],
        [1.2212e+09, 1.1981e+09, 1.2044e+09, 1.1822e+09],
        [1.2685e+09, 1.2445e+09, 1.2511e+09, 1.2281e+09],
        [1.1598e+09, 1.1378e+09, 1.1438e+09, 1.1227e+09],
        [1.2981e+09, 1.2735e+09, 1.2803e+09, 1.2567e+09],
        [1.1644e+09, 1.1424e+09, 1.1484e+09, 1.1272e+09],
        [1.3612e+09, 1.3355e+09, 1.3425e+09, 1.3178e+09],
        [1.2508e+09, 1.2271e+09, 1.2336e+09, 1.2109e+09],
        [1.3706e+09, 1.3446e+09, 1.3517e+09, 1.3268e+09],
        [1.1954e+09, 1.1728e+09, 1.1790e+09, 1.1573e+09],
        [1.2711e+09, 1.2470e+09, 1.2536e+09, 1.2305e+09],
        [1.7219e+09, 1.6893e+09, 1.6982e+09, 1.6669e+09],
        [1.2405e+09, 1.2171e+09, 1.2235e+09, 1.2010e+09],
        [1.3167e+09, 1.2918e+09, 1.2987e+09, 1.2747e+09],
        [1.3192e+09, 1.2943e+09, 1.3011e+09, 1.2771e+09],
        [1.393

tensor([[1.3247e+09, 1.2999e+09, 1.3073e+09, 1.2815e+09],
        [1.4461e+09, 1.4190e+09, 1.4271e+09, 1.3989e+09],
        [1.3981e+09, 1.3719e+09, 1.3797e+09, 1.3524e+09],
        [1.3161e+09, 1.2915e+09, 1.2988e+09, 1.2731e+09],
        [1.3982e+09, 1.3720e+09, 1.3798e+09, 1.3525e+09],
        [1.4430e+09, 1.4160e+09, 1.4241e+09, 1.3959e+09],
        [1.5140e+06, 1.4857e+06, 1.4941e+06, 1.4646e+06],
        [1.3184e+09, 1.2937e+09, 1.3010e+09, 1.2753e+09],
        [1.4517e+09, 1.4245e+09, 1.4326e+09, 1.4043e+09],
        [1.5140e+06, 1.4857e+06, 1.4941e+06, 1.4646e+06],
        [1.5140e+06, 1.4857e+06, 1.4941e+06, 1.4646e+06],
        [1.4238e+09, 1.3972e+09, 1.4051e+09, 1.3773e+09],
        [1.5140e+06, 1.4857e+06, 1.4941e+06, 1.4646e+06],
        [1.5140e+06, 1.4857e+06, 1.4941e+06, 1.4646e+06],
        [1.5140e+06, 1.4857e+06, 1.4941e+06, 1.4646e+06],
        [1.3211e+09, 1.2964e+09, 1.3038e+09, 1.2780e+09],
        [1.5140e+06, 1.4857e+06, 1.4941e+06, 1.4646e+06],
        [1.398

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/jonne/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3437, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-7-4db53a9792d4>", line 2, in <module>
    rvc.fit(data)
  File "<ipython-input-5-191abb44f50f>", line 22, in fit
    train.train(self.model, self.optimizer, train_loader, self.kwargs['loss_func'], self.kwargs['log_step'], self.device)
  File "/home/jonne/Documents/Master_thesis/utils/train_utils.py", line 23, in train
    optimizer.step()
  File "/home/jonne/anaconda3/lib/python3.8/site-packages/torch/optim/optimizer.py", line 88, in wrapper
    return func(*args, **kwargs)
  File "/home/jonne/anaconda3/lib/python3.8/site-packages/torch/autograd/grad_mode.py", line 28, in decorate_context
    return func(*args, **kwargs)
  File "/home/jonne/anaconda3/lib/python3.8/site-packages/torch/optim/adam.py", line 133, in step
    F.adam(params_with_grad,
  File "/home/jo

TypeError: object of type 'NoneType' has no len()